In [ ]:
import plotly.express as px

from random import randrange
import numpy as np
import pandas as pd

import plotly.io as pio
import plotly.graph_objects as go
from pathlib import Path

import plotly.io as pio   
pio.kaleido.scope.mathjax = None # Prevent text box in generated pdf https://github.com/plotly/plotly.py/issues/3469

out_folder = "figures"

def removeLabelsEqualsPart(fig):
  def safeSplit(name):
    names = name.split('=', 1)
    return names[1] if len(names) > 1 else names[0]

  fig.for_each_trace(lambda t: t.update(name =  safeSplit(t.name),
                                      legendgroup =  safeSplit(t.name),
                                      hovertemplate = t.hovertemplate.replace(t.name, safeSplit(t.name))
                                     ) 
                    )
  fig.for_each_annotation(lambda a: a.update(text=a.text.replace("=", " ")))
def removeLabelsColumnName(fig, if_startswith=None):
  def safeSplit(name):
    if if_startswith is not None and name.startswith(if_startswith):
      names = name.split('=', 1)
      return names[1] if len(names) > 1 else names[0]
    else:
      return name


  fig.for_each_trace(lambda t: t.update(name =  safeSplit(t.name),
                                      legendgroup =  safeSplit(t.name),
                                      hovertemplate = t.hovertemplate.replace(t.name, safeSplit(t.name))
                                     ) 
                    )
  fig.for_each_annotation(lambda a: a.update(text=safeSplit(a.text)))  
  


pio.templates["garamond"] = go.layout.Template({'layout': {'font': {'family': 'Garamond'}, 'colorway': px.colors.qualitative.T10}})
pio.templates.default = "plotly_white+garamond"
 
df = pd.read_feather('model_pattern_count.feather')
category_orders_cifar ={"Layer": ["conv2d", "max_pooling2d", "conv2d_1", "max_pooling2d_1", "conv2d_2", "flatten", "dense", "dense_1"],
                        "Input size": [1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]
                }
category_orders_mnist ={"Layer": ["conv2d", "max_pooling2d", "conv2d_1", "max_pooling2d_1", "flatten", "dropout", "dense"],}  
category_orders_inception_v3 ={"Layer": [f"mixed{i}" for i in range(1, 11)] + ["prediction"]} 
category_orders = {"mnist": category_orders_mnist, "cifar10": category_orders_cifar, "inception_v3": category_orders_inception_v3}                  

print(df.describe())            

In [ ]:
px.bar(df[(df["Model"] == "mnist") & (df["Minimum pattern size"] == 5)], x="Input size", y="Number of patterns", facet_row="Model", facet_col="Layer")

In [ ]:
models = ["mnist", "cifar10", "inception_v3"]
layers = {"mnist": "conv2d_1", "cifar10" : "conv2d_2", "inception_v3": "mixed5"}

def plotMinPatternSize(model, layer, data):
    fig = px.bar(data[(data["Model"] == model) & (data["Layer"] == layer) & ((data["Input size"]==10000) | (data["Input size"]==1000)  | (data["Input size"]==5000))], 
                    x="Minimum pattern size", y="Number of patterns", 
                    #facet_col="Layer", 
                    facet_row="Input size", #width=1200, 
                    height=600,
                    category_orders = {"Input size": [1000, 5000, 10000]},
                    title=f"Model: {model}, layer: {layer}")
    fig.update_layout(title_x=0.5)
    #fig.update_yaxes(matches=None)
    #removeLabelsColumnName(fig)
    #fig.show()
    return fig
    
for model in models:
    fig = plotMinPatternSize(model, layers[model], df)
    fig.show()
    fig.write_image(Path(out_folder, f"{model}_min_pattern_size.pdf"))


### Input data size VS Minimum pattern size 

In [ ]:

for model in ["cifar10", "mnist"]:
    dff = df
    dff["Input size"] = dff["Input size"].astype("category")
    cat_orders = category_orders[model]
    cat_orders["Minimum pattern size"] = [5, 6, 7, 8, 9, 10]
    fig = px.bar(dff[dff["Model"] == model], x="Input size", y="Number of patterns", barmode='group', facet_col="Layer", facet_row="Minimum pattern size", 
    category_orders=cat_orders, height=1000)
    removeLabelsColumnName(fig, "Layer")


    fig.write_image(Path(out_folder, f"{model}_min_pattern_size_input_size.pdf"))
    fig.show()

In [ ]:
model = "inception_v3"

fig = px.bar(df[(df["Model"] == model) & (df["Minimum pattern size"] == 5)], x="Input size", y="Number of patterns", barmode='group', facet_col="Layer", category_orders=category_orders[model], height=1000)

#removeLabelsEqualsPart(fig)
#fig.update_xaxes(title_text ="")
#fig.update_yaxes(title_text ="")
fig.show()

In [ ]:
fig = px.bar(df[(df["Model"] == "inception_v3") & (df["Minimum pattern size"] == 5)], x="Input size", y="Number of patterns", barmode='group', facet_col="Layer", height=1000)

#removeLabelsEqualsPart(fig)
#fig.update_xaxes(title_text ="")
#fig.update_yaxes(title_text ="")
fig.show()